In [4]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.schema import Document

# 문서와 메타데이터 준비
documents = [
    Document(page_content="AI는 인류를 도울 수 있습니다.", metadata={"source": "책1", "page": 42}),
    Document(page_content="머신러닝은 데이터에서 학습합니다.", metadata={"source": "책2", "page": 15}),
    Document(page_content="딥러닝은 신경망을 기반으로 합니다.", metadata={"source": "책1", "page": 88}),
]

# 임베딩 모델 및 벡터 데이터베이스 설정
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vector_db = FAISS.from_documents(documents, embedding_model)


In [6]:
# 질문 입력
question = "AI는 무엇인가요?"

# 검색 실행
retrieved_docs = vector_db.similarity_search(question, k=2)
for doc in retrieved_docs:
    print(f"문서 내용: {doc.page_content}")
    print(f"출처: {doc.metadata}")


문서 내용: AI는 인류를 도울 수 있습니다.
출처: {'source': '책1', 'page': 42}
문서 내용: 딥러닝은 신경망을 기반으로 합니다.
출처: {'source': '책1', 'page': 88}


In [7]:
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI

# OpenAI LLM 로드
llm = OpenAI(model="gpt-3.5-turbo", temperature=0)

# RetrievalQA 파이프라인 생성
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vector_db.as_retriever(),
    return_source_documents=True
)

# 질문 처리 및 응답 생성
result = qa_chain({"query": question})

# 결과 출력
print("답변:", result["result"])
print("참조 문서:")
for doc in result["source_documents"]:
    print(f"- 내용: {doc.page_content} | 출처: {doc.metadata}")


/tmp/ipykernel_315887/682250611.py:5: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  llm = OpenAI(model="gpt-3.5-turbo", temperature=0)


ValidationError: 1 validation error for OpenAI
__root__
  Did not find openai_api_key, please add an environment variable `OPENAI_API_KEY` which contains it, or pass `openai_api_key` as a named parameter. (type=value_error)